**Genomics Notebook v1**  
This notebook is part of the materials distributed with [Genomics in the Cloud](https://oreil.ly/genomics-cloud), an O'Reilly book by Geraldine A. Van der Auwera and Brian D. O'Connor. You can read it [online in the O'Reilly library](https://learning.oreilly.com/library/view/genomics-in-the/9781491975183/)
or [order the hardcopy on Amazon](https://www.amazon.com/Genomics-Cloud-GATK-Spark-Docker/dp/1491975199/). The notebook can be used on its own (see configuration notes below); the book provides additional context, instructions and discussion.

**Runtime configuration**   
The exercises in this notebook were developed in a default [Terra](https://app.terra.bio/) runtime environment (Jan 14 2020) corresponding to the container `us.gcr.io/broad-dsp-gcr-public/terra-jupyter-gatk:0.0.13`, complemented by a [startup script](gs://genomics-in-the-cloud/v1/scripts/install_GATK_4130_with_igv.sh) that installs GATK version 4.1.3.0 and an IGV plugin. 

**Kernel**  
By default this notebook opens on a Python 3 kernel. When you have the notebook running in EDIT mode, the upper right corner of the notebook (under the Notebook Runtime widget) should display the label `Python3`. 

----

# Getting Started with Jupyter in Terra
In this section, we run through some exercises to familiarize you with the basic usage of Jupyter notebooks in the Terra environment.


## Running the Hello World Cells
We start with some simple Hello World examples, first in Python, then using magic methods to do the equivalent in R and with a command-line tool call.

### Python Hello World

*Cell 1: Run the basic Hello World in Python*

In [ ]:
print("Hello World!")

*Cell 2: Use a variable to parameterize the greeting*

In [ ]:
greeting = "Hello World!"
print(greeting)

*Cells 3 and 4: Separate the variable declaration and `print` command to demonstrate that state is conserved across cells*

In [ ]:
greeting = "Hello World!"

In [ ]:
print(greeting)

### R Hello World using Python magic methods

*Cell 5: Import the `rpy` package and activate the notebook extension*

In [ ]:
import rpy2
%load_ext rpy2.ipython

*Cell 6: Run the R Hello World with single-line R interpretation using `%R`*

In [ ]:
%R print ("Hello World!")

*Cell 7: Run the R Hello World with a variable and with whole-cell R interpretation using `%%R`*

In [ ]:
%%R
greeting <- "Hello World!"
print(greeting)

### Command-line tool Hello World using Python magic methods

*Cell 8: Run the command-line tool `echo` using `!`*

In [ ]:
! echo "Hello World!"

## Using gsutil to Interact with Google Cloud Storage Buckets
Now that we know how to call command-line tool from the notebook, we look at how to pull in data from GCS buckets 

*Cell 9: List the bucket contents*

In [ ]:
! gsutil ls gs://genomics-in-the-cloud/

*Cell 10: Copy a file from the bucket to the notebook's local storage*

In [ ]:
! gsutil cp gs://genomics-in-the-cloud/hello.txt .

*Cell 11: Read the contents of a locally-stored text file*

In [ ]:
! cat hello.txt

## Setting Up a Variable Pointing to the Germline Data in the Book Bucket
We're going to want to access the data in the bucket multiple times, so we make a variable to avoid hardcoding and repeating file paths.

*Cell 12: Create a Python variable*

In [ ]:
GERM_DATA = "gs://genomics-in-the-cloud/v1/data/germline"

*Cell 13: Use the variable to list the bucket contents*  
*(Erratum: this cell was erroneously numbered 14 in the initial print run)*

In [ ]:
! gsutil ls {GERM_DATA}

*Cell 14: List the `bams` directory to get the paths of the files it contains*

In [ ]:
! gsutil ls {GERM_DATA}/bams

*Cell 15: Copy the BAM file and index for the mother*

In [ ]:
! gsutil cp {GERM_DATA}/bams/mother.ba* .

*Cell 16: List the local working directory to confirm the success of the copy operation*

In [ ]:
! ls .

## Setting Up a Sandbox and Saving Output Files to the Workspace Bucket
Now that we know how to bring in data, let's go over how we're going to save the outputs of any analyses we run.

*Cell 17: Create a new directory, ignoring any errors if the path already exists (`-p`)*

In [ ]:
! mkdir -p sandbox/

*Cell 18: Move the mother BAM and index files that we copied earlier to the sandbox*

In [ ]:
! mv mother.ba* sandbox/

*Cell 19: List the contents of the sandbox to check that everything is where you expect it to be*

In [ ]:
! ls sandbox

*Cell 20: Import the `os` package, look up the value of the `WORKSPACE_BUCKET` environment variable (set by Terra at the kernel level) and store it in a Python variable for easy access*

In [ ]:
import os
WS_BUCKET = os.environ['WORKSPACE_BUCKET']

*Cell 21: Check the value of your new variable*

In [ ]:
print(WS_BUCKET)

*Cell 22: List the full (`-r`) contents of the workspace bucket (results will depend on what other work you have done in your workspace)*

In [ ]:
! gsutil ls -r {WS_BUCKET}

*Cell 23: Copy the contents of your sandbox to the workspace bucket (using `-m` for efficient transfer)*

In [ ]:
! gsutil -m cp -r sandbox {WS_BUCKET}

*Cell 24: List the contents of the copy of your sandbox that is now stored in the bucket*

In [ ]:
! gsutil ls {WS_BUCKET}/sandbox

This completes the "getting started" portion of this notebook.

----

# Visualizing Genomic Data in an Embedded IGV Window
In this section, we embed IGV windows in the notebook in order to visualize genomic data without leaving the notebook environment.

## Setting Up the Embedded IGV Browser
First we need to import the IGV package and initialize a browser window.

*Cell 25: Import the IGV package*

In [ ]:
import igv

*Cell 26: Initialize the browser instance with a genome reference and an interval to focus on (optional)*

In [ ]:
IGV_Explore = igv.Browser(
{"genome": "hg19",
"locus": "chr20:10,025,584-10,036,143"
}
)

*Cell 27: Display the browser window*

In [ ]:
IGV_Explore.show()

## Adding Data to the IGV Browser
Now we can add data by pointing to files in a GCS bucket.

*Cells 28 and 29: Define data tracks for two BAM files (whole genome and exome versions of the mother sample)*

In [ ]:
IGV_Explore.load_track(
{
"name": "Mother WGS",
"url": GERM_DATA + "/bams/mother.bam",
"indexURL": GERM_DATA + "/bams/mother.bai",
"format": "bam"
})

In [ ]:
IGV_Explore.load_track(
{
"name": "Mother Exome",
"url": GERM_DATA + "/bams/motherNEX.bam",
"indexURL": GERM_DATA + "/bams/motherNEX.bai",
"format": "bam"
})

## Setting Up an Access Token to View Private Data
IGV needs an access token to retrieve data from private buckets (including the workspace bucket).

*Cell 30: Emit an acces token and save it to a file*

In [12]:
!gcloud auth print-access-token > token.txt

**Important note:** As long as this file is saved only to your notebook’s local storage, it is secure because
your runtime environment is strictly personal to you and cannot be accessed by others,
even if you share your workspace or your notebook with them. But don’t save this
file to your workspace bucket! Saving it to the bucket would make it visible to anyone
with whom you share the workspace.

*Cell 31: Read the contents of the token file into a Python variable*

In [ ]:
token_file = open("token.txt","r") 
token = token_file.readline()

*Cell 32: Include the token in the track definition of any private files*

In [ ]:
IGV_Explore.load_track(
{
"name": "Workspace bucket copy of Mother WGS",
"url": WS_BUCKET + "/sandbox/mother.bam",
"indexURL": WS_BUCKET + "/sandbox/mother.bai",
"format": "bam",
"oauthToken": token
})

This concludes the section on visualizing genomic data.

----

# Running GATK Commands to Learn, Test, or Troubleshoot
Now let's look at how we can run GATK commands inside the notebook.

## Running a Basic GATK Command: HaplotypeCaller
First we run a simple command like we did in Chapter 5, except we're running directly on the files located in GCS instead of localizing them first.

*Cell 33: Run HaplotypeCaller on files in GCS*

In [ ]:
! gatk HaplotypeCaller \
-R {GERM_DATA}/ref/ref.fasta \
-I {GERM_DATA}/bams/mother.bam \
-O sandbox/mother_variants.200k.vcf.gz \
-L 20:10,000,000-10,200,000

*Cell 34: Verify that the output file is in the sandbox*

In [ ]:
! ls sandbox

**Note:** This works with GATK from anywhere with an internet connection! We could even write the output directly to a GCS bucket if we wanted to; the output filepath just has to start with a valid `gs://` bucket address. 

## Loading the Data (BAM and VCF) into IGV
Now we do a simple visual check of the result.

*Cell 35: Initialize a new IGV window*

In [ ]:
IGV_InspectCalls = igv.Browser(
{"genome": "hg19",
"locus": "chr20:10,002,294-10,002,623"
}
)

IGV_InspectCalls.show()

*Cell 36: Load the variant calls produced by the HaplotypeCaller above*

In [ ]:
IGV_InspectCalls.load_track(
{
"name": "Mother variants",
"url": "files/sandbox/mother_variants.200k.vcf.gz",
"indexURL": "files/sandbox/mother_variants.200k.vcf.gz.tbi",
"format": "vcf"
})

*Cell 37: Load the original BAM file on which you ran HaplotypeCaller*

In [ ]:
IGV_InspectCalls.load_track(
{
"name": "Mother WGS",
"url": GERM_DATA + "/bams/mother.bam",
"indexURL": GERM_DATA + "/bams/mother.bai",
"format": "bam"
})

## Troubleshooting a Questionable Variant Call in the Embedded IGV Browser
Something looks odd so we do some systematic troubleshooting...

*Cell 38: Run HaplotypeCaller on the problem region to produce an output BAM, the `bamout`*

In [ ]:
! gatk HaplotypeCaller \
-R {GERM_DATA}/ref/ref.fasta \
-I {GERM_DATA}/bams/mother.bam \
-O sandbox/motherHCdebug.vcf \
-bamout sandbox/motherHCdebug.bam \
-L 20:10,002,000-10,003,000

*Cell 39: Load the `bamout` file into the IGV window*

In [ ]:
IGV_InspectCalls.load_track(
{
"name": "Mother HC bamout",
"url": "files/sandbox/motherHCdebug.bam",
"indexURL": "files/sandbox/motherHCdebug.bai",
"height": 500,
"format": "bam"
})

This concludes the GATK variant calling section of this notebook. 

----

# Visualizing Variant Context Annotation Data
Finally, let's look at some options for plotting variant annotation data from a callset that has been annotated with metadata from the Genomes in a Bottle (GiaB) truth set.

## Exporting Annotations of Interest with VariantsToTable
First we need to extract annotations from a VCF file to a table format, then load them into an R dataframe.

*Cell 40: Run the GATK tool VariantsToTable to write the annotations of interest to a table to make them easier to work with*

In [ ]:
! gatk VariantsToTable \
-V {GERM_DATA}/vcfs/motherSNP.giab.vcf.gz \
-F CHROM -F POS -F QUAL \
-F BaseQRankSum -F MQRankSum -F ReadPosRankSum \
-F DP -F FS -F MQ -F QD -F SOR \
-F giab.callsets \
-GF GQ \
-O sandbox/motherSNP.giab.txt

*Cell 41: View part of the file to see what the resulting table contains*

In [ ]:
! cat sandbox/motherSNP.giab.txt | head -n300

*Cell 42: Load the table into an R dataframe*

In [ ]:
%%R
library(readr)
motherSNP.giab <- read_delim("sandbox/motherSNP.giab.txt","\t",
escape_double = FALSE,
col_types = cols(giab.callsets = col_character()), trim_ws = TRUE)

## Loading R Script to Make Plotting Functions Available
We provide an R script containing several predefined plotting functions.

*Cell 43: Copy the R script from the bucket to the notebook's working directory, then load it into the R environment*

In [ ]:
! gsutil cp gs://genomics-in-the-cloud/v1/scripts/plotting.R .
%R source("plotting.R")

## Making Density Plots for QUAL by Using makeDensityPlot
We can make density plots to visualize the distribution of a single annotation in our callset.

*Cell 44: Use the predefined function `makeDensityPlot` to visualize the distribution of the QUAL annotation*

In [ ]:
%%R
QUAL_density = makeDensityPlot(motherSNP.giab, "QUAL")
QUAL_density

*Cell 45: Created a second density plot focused on values of QUAL up to 10,000 maximum*

In [ ]:
%%R
QUAL_density_zoom = makeDensityPlot(motherSNP.giab, "QUAL", xmax=10000)
QUAL_density_zoom

*Cell 46: Split the variants into subsets based on the number of callsets they were concordant with in GiaB*

In [ ]:
%%R
QUAL_density_split = makeDensityPlot(motherSNP.giab, "QUAL", xmax=10000,
split="giab.callsets")
QUAL_density_split

**Note:** In this exercise, a higher number of concordant callsets is a likely indicator that a variant call is a true positive rather than an artifact.

## Making a Scatter Plot of QUAL Versus DP
We can make scatter plots to visualize the relationship between two annotations. 

*Cell 47: Use the predefined function `makeScatterPlot` to examine the relationship between QUAL and DP*

In [ ]:
%%R
QUAL_DP_scatterplot = makeScatterPlot(motherSNP.giab, "QUAL", "DP")
QUAL_DP_scatterplot

**Note:** Like above, you can use `xmax` as well as the y-axis equivalent `ymax` to zoom in on ranges of values, and `split` to view the breakdown of the dataa based on the GiaB callsets annotation. 

## Making a Scatter Plot Flanked by Marginal Density Plots
Finally, we can combine the two types of visualizations.

*Cell 48: Use the predefined function `makeScatterPlotWithMarginalDensity` to visualize both annotations' density plots as well as the scatter plot.* 

In [ ]:
%%R
QUAL_DP_comboplot = makeScatterPlotWithMarginalDensity(motherSNP.giab, "QUAL",
"DP", split="giab.callsets", xmax=10000, ymax=100, ptSize=0.5, ptAlpha=0.05)
QUAL_DP_comboplot

This concludes the visualization section of this notebook.

**Important note**  
This particular method of manipulating and visualizing variant data would not scale well for larger datasets, because it involves reading a potentially very large table directly into memory. We chose it for this tutorial because it has the advantage of being approachable
for newcomers, and our primary goal was to give you a sense of the possibilities and familiarize you with the basic mechanics involved. However, for full-scale work,
you’ll probably want to use more robust methods. We recommend checking out [Hail](https://hail.is), a Python-based, genetics-focused toolkit that is extraordinarily scalable and includes a suite of variant quality control functions, among other capabilities. Like some of the more recent GATK tools, Hail is capable of using Spark to parallelize analysis, and has been used to perform genome-wide analysis studies (GWAS) on massive datasets like the UK Biobank. The Terra Library has a few workspaces that feature Hail, including a set of tutorial notebooks and a [complete GWAS example](https://app.terra.bio/#workspaces/amp-t2d-op/2019_ASHG_Reproducible_GWAS-V2).

----

# Epilogue

Whether you're following along with the Genomics in the Cloud book or not, we encourage you to use this notebook as a starting point to experiment with the tools and approaches that we've introduced here. We plan to add more tutorials on [the book's blog](https://broadinstitute.github.io/genomics-in-the-cloud/) so don't hesitate to suggest topics of interest. 